In [1]:
import numpy as np
import re
import pandas as pd
import pickle
import mojimoji
import MeCab
from collections import Counter
from openpyxl import Workbook
from openpyxl import load_workbook

mc = MeCab.Tagger("-Ochasen")

In [2]:
All_df = pd.read_pickle("../data/all.pickle")
print(All_df.shape)
All_df

(12012, 15)


,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


In [3]:
say_id = 0
reply_id = 1
group_id = 2
body = 4


def preprocess(xs, trim_citation=False):
    zs = [""] * len(xs)
    '''半角から全角に'''
    for i, l in enumerate(xs):
        xs[i] = mojimoji.han_to_zen(xs[i])
    '''全角空白'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('　', ' ', str(l))
    '''多重引用'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('＆ｇｔ；', '＞', str(l))
    '''引用部分は除外'''
    count_refer = 0
    for i, l in enumerate(xs):
        sen = ""
        have_refer = 1
        for x in l.split('\n'):
            if len(x) == 0:
                continue
            elif x[0].encode('utf-8') == '＞'.encode('utf-8'):
                have_refer = 1
                x = x[1:]
                if len(x) > 0 and x[0].encode('utf-8') != '＞'.encode(
                        'utf-8') and x[0:2].encode('utf-8') != ' ＞'.encode(
                            'utf-8') and x != ' ':
                    if trim_citation:
                        zs[i] += x + '\n'
                    else:
                        sen = sen + "＞ " + x + '\n'
                continue
            else:
                sen = sen + x + '\n'
        xs[i] = sen.strip()
        count_refer += have_refer
    '''顔文字'''
    pat = re.compile('（[^ぁ-んァ-ン一-龠]+?）', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, ' KAOMOJI ', l)
    '''日本語及び記号{？！ー〜。、} のみ有効'''
    pat = re.compile('[^＞ Ａ-Ｚａ-ｚ０-９ぁ-んァ-ン一-龠？！ー〜。、\n]+?', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, '', l)
    '''記号繰り返し'''
    pat_1 = re.compile('[ｗ]{1,}', re.U)
    pat_2 = re.compile('[？]{1,}', re.U)
    pat_3 = re.compile('[！]{1,}', re.U)
    pat_4 = re.compile('[～]{1,}', re.U)
    pat_5 = re.compile('[ー]{1,}', re.U)
    pat_6 = re.compile('[。]{1,}', re.U)
    for i, l in enumerate(xs):
        l = re.sub(pat_1, 'ｗ', l)
        l = re.sub(pat_2, '？', l)
        l = re.sub(pat_3, '！', l)
        l = re.sub(pat_4, '～', l)
        l = re.sub(pat_5, 'ー', l)
        l = re.sub(pat_6, '。', l)
        xs[i] = l
    if trim_citation:
        return zs
    else:
        return None


def make_xs(orig, trim_citation=False, use_seq2seq=False):
    tmp = np.copy(orig)
    xs = tmp[:, body]
    zs = preprocess(xs, trim_citation=(trim_citation or use_seq2seq))
    if use_seq2seq:
        tmp_prev = np.roll(tmp, 1, axis=0)
        tmp_prev[0, 0] = ""
        ts = tmp[:, [1, 2]]  #　使っていない？？
        xs_prev = tmp_prev[:, body]
        for i in range(len(tmp)):
            if tmp[i, group_id] != tmp_prev[i, group_id]:  # group が異なる
                xs_prev[i] = ""
        preprocess(xs_prev, trim_citation=True)
        for i in range(len(xs)):
#             replay_to = tmp[i, 2]
            replay_to = tmp[i, reply_id]
            if zs[i] != "":
                xs_prev[i] = zs[i]
            elif replay_to != -1:
#                 a = xs[:i][tmp[:i, 1] == replay_to]
                a = xs[:i][tmp[:i, say_id] == replay_to]
                if len(a) != 0:
                    xs_prev[i] = a[-1]
        return xs, xs_prev
    else:
        return xs


def get_doc(xs):
    doc = []
    for ll in xs:
        sen = []
        for l in ll.split('\n'):
            for w in mc.parse(l).split('\n'):
                mx = w.split('\t')
                if len(mx[0]) > 0:
                    sen.append(mx[0])
        doc.append(sen)
    return doc

# テスト

In [4]:
temp = All_df.values[183:194]
print("元データ==========================================")
for t, i in zip(temp, range(len(temp))):
    print(i, " ", t[:5])

xs, xs_pre = make_xs(temp, trim_citation=True, use_seq2seq=True)

doc = get_doc(xs)
doc_pre = get_doc(xs_pre)

print("処理後==========================================")
for t, i in zip(xs, range(len(xs))):
    print(i, " ", [t])
print("処理後(pre)==========================================")
for t, i in zip(xs_pre, range(len(xs_pre))):
    print(i, " ", [t])
print("Mecab==========================================")
for t, i in zip(doc, range(len(doc))):
    print(i, " ", [t])
print("Mecab(pre)==========================================")
for t, i in zip(doc_pre, range(len(doc_pre))):
    print(i, " ", [t])

元データ==========================================
0   [983 885 1234573 'ガーゴ'
 '> 機能についてどの程度のITスキルが必要だと思いますか？\n普段からdropboxとかgoogleドライブを使えれば問題ないレベルかな']
1   [1059 983 1234573 'しんや' '了解しました\n']
2   [1109 -1 1234573 'しんや' 'このテキストで課題提出して大丈夫ですか？']
3   [1163 1109 1234573 'ガーゴ' '> このテキストで課題提出して大丈夫ですか？\n大丈夫ですよ\n']
4   [1207 -1 1234573 'ガーゴ' 'これで終了ですかねお疲れさまでした']
5   [1210 -1 1234573 'しんや' '回答しました。\nありがとうございました']
6   [14 -1 1234574 'ぼ' 'よろしくお願いします\n']
7   [39 -1 1234574 '\\N' 'よろです']
8   [82 -1 1234574 'ソラ' 'よろしくお願いします']
9   [141 -1 1234574 'ぼ' 'これ、3人グループです？\n']
10   [157 -1 1234574 'ソラ' 'グループというよりペアですね']
処理後==========================================
0   ['普段からｄｒｏｐｂｏｘとかｇｏｏｇｌｅドライブを使えれば問題ないレベルかな']
1   ['了解しました']
2   ['このテキストで課題提出して大丈夫ですか？']
3   ['大丈夫ですよ']
4   ['これで終了ですかねお疲れさまでした']
5   ['回答しました。\nありがとうございました']
6   ['よろしくお願いします']
7   ['よろです']
8   ['よろしくお願いします']
9   ['これ、３人グループです？']
10   ['グループというよりペアですね']
処理後(pre)==========================================
0   [' 機能についてどの程度のＩＴスキルが必要だと思いますか？\n']
1   ['普段から

# テキストを処理する

In [5]:
xs, xs_pre = make_xs(All_df.values, trim_citation=True, use_seq2seq=True)
doc = get_doc(xs)
doc_pre = get_doc(xs_pre)
print(type(doc))
print(len(doc))
print(type(doc_pre))
print(len(doc_pre))

<class 'list'>
12012
<class 'list'>
12012


# 処理後の発言データとラベルを結合する

In [6]:
X_col = [
    'sen', 'sen_pre', '16types_a', '16types_b',
    'argument_a', 'argument_b', 'epistemic_a', 'epistemic_b', 'social_a',
    'social_b', 'coordination_a', 'coordination_b'
]

sen = np.asarray(doc).reshape(-1,1)
sen_pre = np.asarray(doc_pre).reshape(-1,1)
X = np.hstack((sen,sen_pre))
labels = All_df.values[:,5:]
X = np.hstack((X,labels))
print(X[0])
print(X[1])
print(X[2])
X_df = pd.DataFrame(X, columns=X_col)

[list(['よろしく', 'お願い', 'し', 'ます', '！', 'EOS']) list(['EOS']) 5 5 21 21 31
 31 40 40 50 50]
[list(['よろしく', 'お願い', 'し', 'ます', 'EOS'])
 list(['よろしく', 'お願い', 'し', 'ます', '！', 'EOS']) 5 5 21 21 31 31 40 40 50 50]
[list(['名前', 'な', 'の', 'が', '恥ずかしい', 'です', 'EOS', 'よろしく', 'お願い', 'し', 'ます', '！', 'EOS'])
 list(['よろしく', 'お願い', 'し', 'ます', 'EOS']) 5 5 21 21 31 31 40 40 50 50]


In [7]:
X_df

,sen,sen_pre,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],5,5,21,21,31,31,40,40,50,50
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",5,5,21,21,31,31,40,40,50,50
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",5,5,21,21,31,31,40,40,50,50
3,"[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",14,14,21,21,31,31,40,40,52,52
4,"[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",1,2,22,22,32,32,41,40,50,50
5,"[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",2,2,22,22,32,32,41,40,50,50
6,"[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",1,1,22,22,32,32,42,42,50,50
7,"[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",2,2,23,23,32,32,41,41,50,50
8,"[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",2,2,25,25,32,32,41,41,50,50
9,"[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",2,2,21,22,31,32,40,41,50,50


# indexを連番に振り直す

In [8]:
X_df_r = X_df.reset_index(drop=True)

# ファイルに保存する

In [9]:
excel_writer = pd.ExcelWriter(
    "../data/all_x.xlsx", engine='xlsxwriter', options={'strings_to_urls': False})
X_df_r.to_excel(excel_writer, "sheet1")
excel_writer.save()
X_df_r.to_pickle("../data/all_x.pickle")